In [1]:
1+2

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


3

In [2]:
2+3

5

In [3]:
def find_valid_r(coeffs, tagdiff):
    p = 2**130 - 5  # Prime modulus
    GF = GF(p)
    possible_rs = []

    # Iterate over possible k values: -4, -3, ..., 4
    for k in range(1):
        print(f"Trying k = {k}")
        rhs = -(tagdiff + k * (2**128)) % p
        
        # Polynomial coefficients: c1 r^5 + c2 r^4 + c3 r^3 + c2 r^2 + c1 r - rhs
        poly = GF(coeffs[0]) * x^5 + GF(coeffs[1]) * x^4 + GF(coeffs[2]) * x^3 \
               + GF(coeffs[3]) * x^2 + GF(coeffs[4]) * x - GF(rhs)
        
        roots = poly.roots(multiplicities=False)
        possible_rs.extend(roots)
    
    # Filter roots based on Poly1305 key constraints
    valid_rs = []
    for r in possible_rs:
        r_int = Integer(r)
        # Convert to 16-byte little-endian
        r_bytes = r_int.to_bytes(16, byteorder='little')
        
        # Check top 4 bits of bytes 3,7,11,15 are clear
        if any((r_bytes[i] & 0xF0) != 0 for i in [3, 7, 11, 15]):
            continue
        
        # Check bottom 2 bits of bytes 4,8,12 are clear
        if any((r_bytes[i] % 4) != 0 for i in [4, 8, 12]):
            continue
        
        valid_rs.append(r_bytes)
    
    return valid_rs



In [471]:
def byte(number, i):
    return (number & (0xff << (i * 8))) >> (i * 8)

def find_valid_r(coeffs, tagdiff):
    p = (1 << 130) - 5  # Prime modulus
    R = PolynomialRing(Integers(p), 'v')  # Define polynomial ring modulo 2^130 - 5
    v = R.gen()
    possible_rs = []

    # Define the polynomial variable

    # Iterate over possible k values: -4, -3, ..., 4
    for k in range(-5, 5):
        # print(f"Trying k = {k}")
        new_coeffs = []
        new_coeffs.extend(coeffs)
        rhs = (tagdiff + k * (1 <<128)) % p
        new_coeffs.append(-rhs)
        new_coeffs.reverse()
        
        # Polynomial coefficients: c1 r^5 + c2 r^4 + c3 r^3 + c2 r^2 + c1 r - rhs
        poly = sum(c * v**i for i, c in enumerate(new_coeffs))
        #print(f"Poly: {poly.polynomial(v)}")
        roots = poly.roots(multiplicities=False)
        # print(roots)
        possible_rs.extend(roots)
    possible_rs = [x.lift() for x in possible_rs]
    for r in possible_rs:
        hex_str = hex(r)[2:].zfill(32)  # Convert to hex, remove '0x', and pad to 32 chars
        formatted_hex = ' '.join(hex_str[i:i+2] for i in range(0, len(hex_str), 2))
        print(f"{str(r)}\t: {formatted_hex}")

    possible_rs = [
        r for r in possible_rs 
        if  (byte(r, 3)  & 0xF0 == 0) 
        and  (byte(r, 7)  & 0xF0 == 0)  
        and  (byte(r, 11)  & 0xF0 == 0)  
        and  (byte(r, 15)  & 0xF0 == 0)  
        and  (byte(r, 4)  & 0x03 == 0)  
        and  (byte(r, 8)  & 0x03 == 0)  
        and  (byte(r, 12)  & 0x03 == 0)  
    ]
    return possible_rs

def find_valid_s(r_candidates, coeffs1, coeffs2, tag1, tag2):
    p = (1 << 130) - 5  # Prime modulus
    rs_candidates = []
    print(r_candidates)
    for r in r_candidates:
        acc1 = 0
        acc2 = 0
        for i in range(len(coeffs1)):
            acc1 += coeffs1[i] % p
            acc1 = (acc1 * r) % p
            acc2 += coeffs2[i] % p
            acc2 = (acc2 * r) % p
        #print(acc1, acc2)
        s1 = (tag1 - acc1) % (1 << 128)
        s2 = (tag2 - acc2) % (1 << 128)
        if s1 == s2:
            rs_candidates.append([r, s1])
    return rs_candidates

In [491]:
coeffs1 = [538640274081240381019147370272758415192, 462029317561921617150287947377116959195, 514826131592740245787887153031584711298, 535550505289509202810815754775617154302, 340282369155093552765040269583743790077, 340282366920938464883773901107403685888]
coeffs2 = [518575531808390145367352643422387617352, 439370050837056053909361816896710739392, 613210017993548771521380264009051921874, 550280077919642477568980272650752502763, 340282366920938463463374621277054575026, 340282366920938464754646692591436824576]
diff_coeffs = [20064742272850235651794726850370797840, 22659266724865563240926130480406219803, -98383886400808525733493110977467210576, -14729572630133274758164517875135348461, 2234155089301665648306689215051, 129127208515966861312]
tag1 = 214997742450230526426862784708746195147
tag2 = 11700909561346116411883985678737627548
tagdiff = 203296832888884410014978799030008567599

In [492]:
find_valid_s(find_valid_r(diff_coeffs, tagdiff), coeffs1, coeffs2, tag1, tag2)

672756700999113330458682122112888731074	: 1f a2 03 a4 24 c7 0a 67 70 9a 21 38 d7 45 3f 5c 2
910798585191932512621484758543117777517	: 2a d3 56 d4 63 76 5b 27 50 05 f9 1c 56 5f 2f a6 d
884041724949607071490947755470871379759	: 29 91 43 e3 65 9a b2 fa a6 24 0f be 33 a0 0b f2 f
1163984120322327139423653785278369166938	: 36 ba f2 fe d2 02 ba 1d 16 bd c1 59 b1 aa ef 65 a
9891123961954051960689791493241080567	: 07 70 f6 10 09 57 15 2c 0a 95 36 ec 0e 98 8a f7
930170876911270808337057867339014468167	: 2b bc 86 4f f8 6f 3e 4e c3 67 b7 6c 43 3b 1f 24 7
149957928990057049356322976306767414381	: 70 d0 d8 b9 bb 0a d1 19 c3 a2 ac 3d d0 6d 44 6d
940055141728703913411651768454398339321	: 2c 33 80 8e 0f d0 48 ce 35 48 4b 2b d9 56 3e 8f 9
972587388263851485379169609780609383084	: 2d bb 18 47 83 2a 20 7f 5c 47 a8 74 18 4d 37 aa c
142974882975895100839682454143618919025	: 6b 8f f5 f9 bb 52 5b 9c 8d 44 5f 3c 0c 22 2a 71
[9891123961954051960689791493241080567]


[[9891123961954051960689791493241080567,
  88048986216729752815100368924182949598]]

In [476]:
def clamp(r):
    return r & 0x0ffffffc0ffffffc0ffffffc0fffffff